In [1]:
from gensim.models import Word2Vec
import numpy as np
from pprint import pprint
from konlpy.tag import Komoran;
k = Komoran()  
import nltk  
from nltk import FreqDist 
import pandas as pd
import codecs

C:\Users\Jongil Park\AppData\Local\conda\conda\envs\python_study\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


첫번째 테스트 케이스
- '교통사고'키워드로 검색한 판례 375개를 Word2Vec 모델의 데이터 셋으로 활용한 테스트 케이스1.

두번째 테스트 케이스(2271개 판례 활용)
- 5개의 카테고리(교통사고, 교통, 사고, 운전, 차량)를 사용하여 Word2Vec 모델의 데이터 셋으로 활용한 테스트 케이스2.


이 코드는 첫번째 테스트 케이스의 내용으로 구성하였습니다.

데이터 전처리 작업 및 
사용자의 입력값(Ex. 도로에서 음주 후 음주운전 중에 사고가 났고, 당시 알콜 농도는 0.15였습니다)을 토크나이징하여
단어값들과 가장 연관성이 높은 Word2Vec모델의 값을 이용할 예정이며, 그 값이 높을수록 해당 판례와 관련성이 많다고 가정하였습니다.
때문에 각 판례들을 각각 파싱하여 단어 구성과 단어의 갯수를 딕셔너리로 구성합니다 - "1차 완료"부분

'교통사고'판례에서 나온 단어들을 토크나이징하고 그 중 사용할 단어와 사용하지 않을 단어 셋을 구성하였고,
사용하지 않을 단어를 제외한 Word2Vec모델에 넣을 단어 말뭉치를 구성하였습니다 - "2차 완료"부분

In [3]:
# 1차적으로 사용하지 않을 키워드, 판례에서 필요한 부분(판례 제목, 사건 번호 등을 따로 뽑아 놓음)
fileName_dnusing_wordSet = 'C:/Users/Jongil Park/PycharmProjects/ai_study/New_projectDirectory_law2' \
                           '/DataSet/Gyotongsago_data_375/Gyotongsago_dataSet/Gyotongsago_dnUsingWordSet_375_onlyNoun.txt'
fileName_title = 'C:/Users/Jongil Park/PycharmProjects/ai_study/New_projectDirectory_law2' \
                 '/DataSet/Gyotongsago_data_375/Gyotongsago_dataSet/Gyotongsago_lawTitle_375.txt'
fileName_keyNum = 'C:/Users/Jongil Park/PycharmProjects/ai_study/New_projectDirectory_law2' \
                  '/DataSet/Gyotongsago_data_375/Gyotongsago_dataSet/Gyotongsago_lawNumber_375.txt'

## 불러온 파일 리스트화
with open(fileName_dnusing_wordSet, 'r') as infile:
    word_list = [line.rstrip() for line in infile]

## 타이틀만 모아놓은 것 리스트로 만들기
with open(fileName_title, 'r') as infile2:
    title_list = [line.rstrip() for line in infile2]

## 키워드 넘버만 모아놓은 것 리스트 만들기
try:
    with open(fileName_keyNum) as infile3:
        keyNum_list = [line.rstrip() for line in infile3]
except UnicodeDecodeError:   ## TODO: 텍스트 파일을 불러올 때, 에러가 나는 이유는 무엇인가? 왜 그래서 codecs를 써야 하는가?
    with codecs.open(fileName_keyNum, "r", "utf-8") as infile3:
        keyNum_list = [line.rstrip() for line in infile3]


## 여기서 pasing 된 corpus 값을 명사만 추출하여 각 단어별 갯수를 딕셔너리로 만들어주고 append
def append_noun_words(corpus):
    noun_words = ['NNG', 'NNB', 'NP']  # 일반명사, 고유명사, 대명사만 학습  // 이렇게 한 이유에 대해서?
    results = []
    for text in corpus:
        for noun_word in noun_words:
            if noun_word in text[1]:
                results.append(text[0])
    return results

## {판례의 명사 키워드의 단어:갯수 매칭된 리스트}
total_panrye_parsingReason = []  # 이 안의 값은 딕셔너리가 되어야 함

## '교통사고' 하나의 카테고리만 활용할 때,
file_name = ['C:/Users/Jongil Park/PycharmProjects/ai_study/New_projectDirectory_law2/DataSet/Gyotongsago_data_375'
             '/Gyotongsago_dataSet/Gyotongsago_reason_375/({0}).txt']
file_len = [375]

for z in range(len(file_name)):

    for i in range(file_len[z]):    # len(title_list)
        ione_panrye_parsingReason = {}

        try:
            with open(file_name[z].format(i+1), 'r') as f:
                texts = f.read()
                corpus = k.pos("\n".join([s for s in texts.split("\n") if s]))
        except UnicodeDecodeError:
            with codecs.open(file_name[z].format(i+1), "r", "utf-8") as f:
                texts = f.read()
                corpus = k.pos("\n".join([s for s in texts.split("\n") if s]))

        corpus = append_noun_words(corpus)
        corpus_ko = nltk.Text(corpus, name="각 판례별 명사 처리")
        corpus_ko_vocab = corpus_ko.vocab()
        corpus_vocab_common = corpus_ko_vocab.most_common(len(corpus_ko_vocab))

    ## 여기에 필요한 키워드 값만 추출하는 것을 추가 할 것.
        for j in range(len(corpus_vocab_common)):
            if corpus_vocab_common[j][0] not in word_list:
                ione_panrye_parsingReason[corpus_vocab_common[j][0]] = corpus_vocab_common[j][1]
        total_panrye_parsingReason.append(ione_panrye_parsingReason)

print("1차 완료: 각 타이틀, 사건번호, 딕셔너리(단어: 번호)매칭")

1차 완료: 각 타이틀, 사건번호, 딕셔너리(단어: 번호)매칭


In [4]:
#======================================================================================================
## TODO: 3번: W2V을 위한 데이터 전처리, '기준'을 어떻게 둘 것인가?
"""최종적으로 데이터 프레임화를 위한 리스트"""
# TODO: 판례 데이터 사용방법 
# 1. '교통사고'의 이유 부분 전체 합을 이용
# 2. '교통사고' 각 판례(375개)를 각각 W2V 모델에 사용하고 그 값들을 활용?

# 각 교통사고 관련 카테고리의 '이유' 부분의 sum 값을 활용
data_file = ['C:/Users/Jongil Park/PycharmProjects/ai_study/New_projectDirectory_law2/DataSet/Gyotongsago_data_375'
             '/Gyotongsago_dataSet/Gyotongsago_totalSum_375.txt']

total_sumWord = []
for i in range(len(data_file)):
    try:
        with open(data_file[i]) as f:
            texts = f.read()
            corpus = k.pos("\n".join([s for s in texts.split("\n") if s]))  # 이것의 의미 다시 한번 생각

    except UnicodeDecodeError:
        with codecs.open(data_file[i], "r", "utf-8") as f:
            texts = f.read()
            corpus = k.pos("\n".join([s for s in texts.split("\n") if s]))

    corpus = append_noun_words(corpus)
    corpus_ko = nltk.Text(corpus, name="각 판례별 명사 처리")
    corpus_ko_vocab = corpus_ko.vocab()
    corpus_ko_vocab_items = corpus_ko_vocab.items()
    corpus_ko_vocab_items = list(corpus_ko_vocab_items)  ## TODO: 과연 items순이 옳은가?
    #corpus_vocab_common = corpus_ko_vocab.most_common(len(corpus_ko))

    ## new_corpus 는 살릴 단어가 하나씩 밖에 안 들어가 있다. 그 개수가 빠져있는 합.
    new_corpus = []
    corpus_values = []
    new_corpus2 = []

    for z in range(len(corpus_ko_vocab_items)):
        corpus_word = corpus_ko_vocab_items[z][0]
        if corpus_word not in word_list:
            new_corpus.append([corpus_word])
            corpus_values.append(corpus_ko_vocab_items[z][1])
    new_corpus2 = sum(new_corpus, [])

    last_using_word = []
    for j in range(len(new_corpus)):
        last_using_word.append(''.join(new_corpus[j]))   # join => 리스트를 문자열로 합치기
    """
    df3 = pd.DataFrame(corpus_values, index=last_using_word)
    df3.to_excel("C:/Users/user/PycharmProjects/python_study/projectDirectory/POS&embedding/test_dataset/" \
                 "TOTAL_GYOTONGSAGO_DATA/TOTAL_GYOTONGSAGO_USING_WORD_LAST({0}).xlsx".format(i+1),
                 sheet_name='단어')
    print("TOTAL_GYOTONGSAGO_USING_WORD_LAST 저장완료")
    """
    
    ## 이것이 corpus 에서 쓰지 않은 키워드를 갯수만큼 제외시키는 것
    word_corpus_last = [x for x in corpus if x not in word_list]
    word_corpus_last2 = []
    for z in range(len(word_corpus_last)):
        word_corpus_last2.append([word_corpus_last[z]])   # word_corpus_last2.append(word_corpus_last[z].split())에서 변경

    total_sumWord.append(word_corpus_last2)
    # total_sumWord.append(new_corpus)
total_sumWord = sum(total_sumWord, [])
df4 = pd.DataFrame(total_sumWord)
df4.to_excel('C:/Users/Jongil Park/PycharmProjects/ai_study/New_projectDirectory_law2/DataSet/Gyotongsago_data_375/'
             'Gyotongsago_resultSet/total_sumWord_for_W2V.xlsx',
             sheet_name='최종W2V단어말뭉치')

print("word2vec 전, 2차 완료(Word2Vec모델에 넣을 단어 말뭉치 형성)")
print("len(total_sumWord): ", len(total_sumWord))

word2vec 전, 2차 완료(Word2Vec모델에 넣을 단어 말뭉치 형성)
len(total_sumWord):  63352


Word2Vec모델에 넣을 최종 63352개의 단어 말뭉치 확보
(각 단어의 중복 및 순서 혀용)

In [5]:
## Word2Vec
word2vec_model = Word2Vec(total_sumWord, size=200,     # 인자값에 new_corpus 혹은 word_corpus_last2
                          window=2,
                          min_count=10,
                          workers=4,
                          iter=10000, sg=1)

w2v_key = word2vec_model.wv.vocab.keys()
input_keyword_last = ['테스트를 위한 빈값 넣어놓은 것']
print("len(w2v_key):", len(w2v_key))

len(w2v_key): 596


Word2Vec모델을 통해 나온 중복 제외 최종 키워드 갯수: 596

In [6]:
def input_to_keyword():
    input_text = input("검색어를 입력하세요(끝내시려면 enter 키를 눌러주세요): ")
    k = Komoran()    # 새로 변수를 정의하지 않을시 반복해서 사용자가 검색어를 입력하면 에러가 뜨는 경우가 발생
    if input_text is '':
        input_keyword_last = None
        return input_keyword_last
    else:
        input_corpus = k.pos("\n".join([s for s in input_text.split("\n") if s]))
        parsed_input = append_noun_words(input_corpus)
        input_keyword_last = []
        for j in range(len(parsed_input)):
            # if parsed_input[j] not in word_list:
            # 위에 문장을 쓰지 않는 이유는 사용자가 검색한 키워드가 항상 우리 키워드 안에 있지는 않지만, 사용자의 키워드를 보여줄 필요는 있다.
            input_keyword_last.append(parsed_input[j])
        return input_keyword_last

repeat_num = 1    # 저장할 액셀 파일에 숫자를 붙여주기 위한 기초값, 첫번째를 뜻함.

while True:
    input_keyword_last = input_to_keyword()  # 인풋값을 사용할 키워드로 정리한 값
    if input_keyword_last is None:
        break
    columns_keyword = "·".join(input_keyword_last)  # 컬럼 이름을 위한 합치기

    df = pd.DataFrame()
    predict_words = []
    predict_values = []

    ## TODO: 과연 내가 찾으려고 하는 연관성이 predict_output_word()가 맞는가?
    predict_model = word2vec_model.predict_output_word(input_keyword_last, topn=len(w2v_key))
    for n in range(len(predict_model)):
        predict_words.append((predict_model[n][0]))
        predict_values.append(predict_model[n][1])
    val_words = pd.Series(predict_words)
    val_values = pd.Series(predict_values)
    df[columns_keyword] = val_words
    df['vector값'] = val_values
    #word2vec_model.max_final_vocab  # 이것은 왜 필요한거지? 아마 그냥 들어간거 같은데!!??

    df.to_csv('C:/Users/Jongil Park/PycharmProjects/ai_study/New_projectDirectory_law2/DataSet/Gyotongsago_data_375/Gyotongsago_resultSet'
              '/Gyotongsago_375_W2V_predict({0}).csv'.format(repeat_num),
              encoding='CP949')

    print("len(predict_model): ", len(predict_model))
    print("({0})차 구동완료".format(repeat_num))

    ## 키워드 연관 단어 상위 20개를 기준으로 한 카운트 값
    df_top20 = df[columns_keyword].iloc[:20]   ## iloc와 loc 차이? 여기에선 같은 값이 나오지만 둘의 차이 명확히 알기
    count_list_total = []
    for kk in range(len(total_panrye_parsingReason)):
        count = 0
        for j in range(len(df_top20)):
            if df_top20[j] in total_panrye_parsingReason[kk]:
                # 현재 count 값은 상위 20개를 순서상관없이 + 값, 순서별로 가중치를 주는 방법도 고려할 것.
                count += total_panrye_parsingReason[kk][df_top20[j]]
        count_list_total.append(count)

    # 카운트 값이 높은 순서의 인덱스 값의 리스트를 만들기
    df_count = pd.DataFrame()
    val_title_list = pd.Series(title_list)
    val_keyNum_list = pd.Series(keyNum_list)

    df_count = pd.DataFrame({(columns_keyword): count_list_total})
    df_count['판례이름'] = val_title_list
    df_count['사건번호'] = val_keyNum_list
    last_df_count = df_count.sort_values(by=columns_keyword, ascending=False)

    last_df_count.to_csv('C:/Users/Jongil Park/PycharmProjects/ai_study/New_projectDirectory_law2/DataSet'
                         '/Gyotongsago_data_375/Gyotongsago_resultSet/Gyotongsago_375_W2V_result({0}).csv'.format(repeat_num),
                         encoding='CP949')
    repeat_num += 1

print("구동완료")

검색어를 입력하세요(끝내시려면 enter 키를 눌러주세요): 도로에서 음주 후 운전 중에 사고가 났습니다. 혈중알콜농도는 0.15정도 였습니다.
len(predict_model):  596
(1)차 구동완료
검색어를 입력하세요(끝내시려면 enter 키를 눌러주세요): 골목길에서 좌회전 도중 보행자를 차로 치었습니다. 과실 부분은 어떻게 될까요?
len(predict_model):  596
(2)차 구동완료
검색어를 입력하세요(끝내시려면 enter 키를 눌러주세요): 도로에서 직진 중에 좌회전 차량과 접촉 사고가 났습니다.
len(predict_model):  596
(3)차 구동완료
검색어를 입력하세요(끝내시려면 enter 키를 눌러주세요): 
구동완료


In [9]:
df.iloc[:20]

,도로·직진·중·좌회전·차량·접촉·사고,vector값
0,피고인,0.001678
1,반대편,0.001678
2,취득,0.001678
3,유인,0.001678
4,차례,0.001678
5,계약,0.001678
6,앞쪽,0.001678
7,청구,0.001678
8,무시,0.001678
9,공동정범,0.001678


예를 들어, 사용자의 입력이
"도로에서 직진 중에 좌회전 차량과 접촉 사고가 났습니다."일 때,
해당 입력값을 파싱하여 Word2Vec모델에 넣고 가장 연관성이 높다고(높은 벡터값) 나온 단어 셋을 만들었습니다.

In [12]:
last_df_count.iloc[:20]

,도로·직진·중·좌회전·차량·접촉·사고,판례이름,사건번호
354,245,폭력행위등 처벌에 관한 법률위반(공동공갈)·업무 방해·공갈·폭력행위등 처벌에 관한 ...,2011노163
36,176,살인(예비적죄명:교통사고처리특례법위반)·사기,2015노358
12,158,살인(예비적죄명:교통사고처리특례법위반)·사기(남편이 보험금을 노리고 교통사고를 내어...,2017도1549
86,133,강도치상·특수강도·도로교통법위반(무면허운전)·사기·교통사고처리특례법위반,2006고합880
353,131,폭력행위등 처벌에 관한 법률위반(공동 공갈)·업무 방해·공갈·폭력행위등 처벌에 관한...,2009고합613
373,129,"감금치사[선택적죄명:살인,인정된죄명:도로교통법위반(사고후미조치)·유기치사]",2013노2492
15,115,특정범죄가중처벌등에관한법률위반(도주차량)·교통사고처리특례법위반·도로교통법위반·도로교...,2006노2898
85,101,강도치상(인정된죄명:절도·상해)·특수강도·도로교통법위반(무면허운전)·사기·교통사고처...,2007노193
340,96,특정범죄가중처벌등에관한법률위반(영리약취·유인등)·아동·청소년의성보호에관한법률위반(강...,2011노573
301,87,교통사고처리특례법위반,93노4273


위에서 나온 벡터값이 높은 단어 상위 20개를 활용하여, 각 판례에 20개의 단어 들어있는 갯수대로 카운트를 매겼습니다.
위에 가정한대로 카운트 값이 높을수록 해당 판례가 가장 입력값과 연관성이 높은 판례라고 생각했습니다.
위에서 확보해놓은 각 판례의 {단어: 갯수}딕셔너리를 통하여 카운트 값이 가장 큰 값 순으로 정렬하였습니다.

DataFrame의 내용은
맨 왼쪽의 숫자는 1-375개의 판례의 순서 번호이며,
중간 숫자는 해당 키워드('도로·직진·중·좌회전·차량·접촉·사고')의 카운트값
카운트값이 높은 순서대로 판례이름ㅁ과 해당 사건번호 순서입니다.